In [1]:
%%capture

import pandas as pd
import geopandas as gpd
from calitp_data_analysis.sql import query_sql
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from calitp_data_analysis.geography_utils import CA_NAD83Albers_m, WGS84
import calitp_data_analysis.magics
gcsgp = GCSGeoPandas()
from shared_utils import gtfs_utils_v2, rt_utils

import utils
import update_vars
import altair as alt
import numpy as np
import shapely
import branca

import importlib
importlib.reload(utils)
importlib.reload(update_vars)

# Passenger Flow and Ridership Charts by Route and Origin/Destination

* Hues represent origin stops (and may repeat where there are many)
* Within each origin group, darker shades represent longer O/D pairs
* Currently based on April 2025 figures only

In [2]:
analysis_date = update_vars.ANALYSIS_DATE

In [3]:
ca = gcsgp.read_parquet('gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/ca_boundary.parquet')

In [4]:
feeds = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(analysis_date)

In [5]:
sanj = feeds.query('name == "Amtrak San Joaquins Schedule"')

In [6]:
sanj_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=sanj.feed_key, shape_cols=update_vars.shape_cols)
sanj_shapes = sanj_shapes.to_crs(CA_NAD83Albers_m)
sanj_shapes = sanj_shapes.assign(length_meters = sanj_shapes.geometry.length)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(


In [7]:
sanj_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=sanj.feed_key, trip_cols=update_vars.trip_cols).query('route_type == "3"')

In [8]:
sanj_stops = gtfs_utils_v2.get_stops(analysis_date, operator_feeds=sanj.feed_key, stop_cols=update_vars.stop_cols).to_crs(CA_NAD83Albers_m)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


In [9]:
sanj_st = gtfs_utils_v2.get_stop_times(analysis_date, operator_feeds=sanj.feed_key, get_df=True,
                                      trip_df = sanj_trips, stop_time_cols=update_vars.stop_time_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(


In [10]:
bus_st = sanj_st.merge(sanj_trips[['trip_id', 'direction_id', 'route_id', 'route_short_name',
                                   'shape_array_key', 'shape_id']], on='trip_id')

In [11]:
sanj_shapes_trip_info = (sanj_shapes.merge(sanj_trips.drop_duplicates(subset=['shape_array_key']), on = ['shape_array_key', 'feed_key'])
                            .drop(columns=['base64_url', 'regional_feed_type', 'block_id',
                                           'route_desc', 'agency_id', 'network_id',
                                          'route_key'])
                        )

In [12]:
sort_cols = ['route_id', 'direction_id']

In [13]:
def longest_by_route_dir(shape_trip_df):
    #  https://github.com/cal-itp/data-analyses/blob/1abcd5d05163176567f644f6f245366dd9cc53fc/rt_segment_speeds/segment_speed_utils/gtfs_schedule_wrangling.py#L426
    df = (shape_trip_df.query('route_type == "3"')
     .sort_values(sort_cols + ['length_meters'], ascending=[True for i in sort_cols] + [False])
     .drop_duplicates(subset=sort_cols)
     .reset_index(drop=True)
    )
    return df

In [14]:
sanj_long_shapes = longest_by_route_dir(sanj_shapes_trip_info)

In [15]:
source_ridership = pd.read_excel(update_vars.RIDERSHIP_PATH)
source_ridership = source_ridership.assign(od = source_ridership.orig + '->' + source_ridership.dest)
source_ridership = source_ridership.assign(route_short_name = source_ridership.ca_bus_route.str.replace('Rt', 'Route'))
source_ridership = source_ridership.query('trip_year == 2025 & trip_month == 4') ## TODO trends, etc

rider_to_gtfs_dict = {'Route 1A': 'Route 1', 'Route 1B': 'Route 1', 'Route 1C': 'Route 1c',
                     'Route 20 - B': 'Route 20', 'Route 3R': 'Route 3'}
strip_zero = lambda route_str: ' '.join([x.lstrip('0') for x in route_str.split(' ')])
rider_to_gtfs = lambda route_str: rider_to_gtfs_dict[route_str] if route_str in rider_to_gtfs_dict.keys() else route_str

source_ridership = source_ridership.assign(route_short_name = source_ridership.route_short_name.map(strip_zero).map(rider_to_gtfs))

In [16]:
bus_stops = sanj_st.stop_id.unique()

In [17]:
get_ridership_unique_stops = lambda df: np.union1d(df.orig, df.dest)

In [18]:
rider_stops = get_ridership_unique_stops(source_ridership)

In [19]:
sanj_bus_to_rider_stops = dict(zip(bus_stops, [x[1:] for x in bus_stops]))
sanj_bus_to_rider_stops['bLOS'] = 'LAX'

In [20]:
bus_st = pd.merge(sanj_stops[['stop_id', 'geometry']], bus_st, on='stop_id')
bus_st = bus_st.assign(amtrak_stop = bus_st.stop_id.map(sanj_bus_to_rider_stops)).sort_values(['trip_id', 'stop_sequence'])

In [23]:
# pd.Series(sanj_bus_to_rider_stops.values()).isin(pd.Series(rider_stops)).value_counts()

# gtfs_no_rider_data = pd.Series(sanj_bus_to_rider_stops.values())[~pd.Series(sanj_bus_to_rider_stops.values()).isin(pd.Series(rider_stops))]
# gtfs_no_rider_data #  OK -- all either served by RABA/YARTS or outside CA...

## Route 1

In [24]:
rt1_test = sanj_long_shapes.query('route_id == "1"')

In [25]:
rt1_test[['name', 'route_short_name', 'route_long_name', 'shape_id']]

,name,route_short_name,route_long_name,shape_id
0,Amtrak San Joaquins Schedule,Route 1,Fresno/Bekersfield - Los Angeles- SantaAna - O...,jfy7
1,Amtrak San Joaquins Schedule,Route 1,Fresno/Bekersfield - Los Angeles- SantaAna - O...,xurp


In [26]:
import chart_utils
importlib.reload(chart_utils)

<module 'chart_utils' from '/home/jovyan/data-analyses/thruway_intercity_bus/chart_utils.py'>

In [27]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt1_test.iloc[0], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 1A')

In [28]:
chart

alt.Chart(...)

In [29]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt1_test.iloc[1], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 1A')

In [30]:
chart

alt.Chart(...)

## Route 1b

In [31]:
rt1b_test = sanj_long_shapes.query('route_id == "1"')

In [32]:
rt1b_test[['name', 'route_short_name', 'route_long_name', 'shape_id']]

,name,route_short_name,route_long_name,shape_id
0,Amtrak San Joaquins Schedule,Route 1,Fresno/Bekersfield - Los Angeles- SantaAna - O...,jfy7
1,Amtrak San Joaquins Schedule,Route 1,Fresno/Bekersfield - Los Angeles- SantaAna - O...,xurp


In [33]:
import chart_utils
importlib.reload(chart_utils)

<module 'chart_utils' from '/home/jovyan/data-analyses/thruway_intercity_bus/chart_utils.py'>

In [34]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt1b_test.iloc[0], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 1B')

In [35]:
chart

alt.Chart(...)

In [36]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt1b_test.iloc[1], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 1B')

In [37]:
chart

alt.Chart(...)

## Route 1C

In [38]:
rt1c_test = sanj_long_shapes.query('route_id == "1c"')

In [39]:
rt1c_test[['name', 'route_short_name', 'route_long_name', 'shape_id']]

,name,route_short_name,route_long_name,shape_id
14,Amtrak San Joaquins Schedule,Route 1c,Bakersfield to Los Angeles/Santa Monica,wo7l
15,Amtrak San Joaquins Schedule,Route 1c,Bakersfield to Los Angeles/Santa Monica,r66l


In [40]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt1c_test.iloc[0], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 1C')

correcting LAX to SMN for Route 1c


In [41]:
chart

alt.Chart(...)

In [42]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt1c_test.iloc[1], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 1C')

correcting LAX to SMN for Route 1c


In [43]:
chart

alt.Chart(...)

## Route 3

* Operated by RABA Chico - Redding, seems to create a data gap in that section

In [44]:
rt3_test = sanj_long_shapes.query('route_id == "3"')

In [45]:
rt3_test[['name', 'route_short_name', 'route_long_name', 'shape_id']]

,name,route_short_name,route_long_name,shape_id
20,Amtrak San Joaquins Schedule,Route 3,Redding Sacramento Stockton,1wk1
21,Amtrak San Joaquins Schedule,Route 3,Redding Sacramento Stockton,1vos


In [53]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt3_test.iloc[0], stop_times=bus_st, ridership=source_ridership,
                              # ridership_data_route='Rt 03'
                                                  )

In [54]:
chart

alt.Chart(...)

In [55]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt3_test.iloc[1], stop_times=bus_st, ridership=source_ridership,
                              # ridership_data_route='Rt 03'
                                                  )

In [56]:
chart

alt.Chart(...)

## Route 6

In [ ]:
rt6_test = sanj_long_shapes.query('route_id == "6"')

In [ ]:
rt6_test[['name', 'route_short_name', 'route_long_name', 'shape_id']]

In [ ]:
import chart_utils
importlib.reload(chart_utils)

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt6_test.iloc[0], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 06')

In [ ]:
chart

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt6_test.iloc[1], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 06')

In [ ]:
chart

## Route 7

In [ ]:
rt7_test = sanj_long_shapes.query('route_id == "7"')

In [ ]:
rt7_test[['name', 'route_short_name', 'route_long_name', 'shape_id']]

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt7_test.iloc[0], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 07')

In [ ]:
chart

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt7_test.iloc[1], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 07'
                                                  )

In [ ]:
chart

## Route 10

* Seems to function as two halves with train connection in the middle, but a few passengers can and do ride through (for example Santa Barbara - Las Vegas)

In [ ]:
rt10_test = sanj_long_shapes.query('route_id == "10"')

In [ ]:
rt10_test[['name', 'route_short_name', 'route_long_name', 'shape_id']]

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt10_test.iloc[0], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 10'
                                                  )

In [ ]:
chart

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt10_test.iloc[1], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 10'
                                                  )

In [ ]:
chart

## Route 15a (YARTS)

* some sort of bug on 15b, 15 (both also YARTS, will check later)

In [ ]:
rt15_test = sanj_long_shapes.query('route_id == "15a"')

In [ ]:
rt15_test[['name', 'route_short_name', 'route_long_name', 'shape_id']]

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt15_test.iloc[0], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 15'
                                                  )

In [ ]:
chart

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt15_test.iloc[1], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 15'
                                                  )

In [ ]:
chart

## Route 18

In [ ]:
rt18_test = sanj_long_shapes.query('route_id == "18"')

In [ ]:
rt18_test[['name', 'route_short_name', 'route_long_name', 'shape_id']]

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt18_test.iloc[0], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 18'
                                                  )

In [ ]:
chart

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt18_test.iloc[1], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 18'
                                                  )

In [ ]:
chart

## Route 19

In [ ]:
rt19_test = sanj_long_shapes.query('route_id == "19"')

In [ ]:
rt19_test[['name', 'route_short_name', 'route_long_name', 'shape_id']]

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt19_test.iloc[0], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 19')

In [ ]:
chart

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt19_test.iloc[1], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 19')

In [ ]:
chart

## Route 20

In [ ]:
rt20_test = sanj_long_shapes.query('route_id == "20"')

In [ ]:
rt20_test[['name', 'route_short_name', 'route_long_name', 'shape_id']]

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt20_test.iloc[0], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 20')

In [ ]:
chart

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt20_test.iloc[1], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 20')

In [ ]:
chart

## Route 20c

In [ ]:
rt20c_test = sanj_long_shapes.query('route_id == "20c"')

In [ ]:
rt20c_test[['name', 'route_short_name', 'route_long_name', 'shape_id']]

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt20c_test.iloc[0], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 20'
                                                  )

In [ ]:
chart

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt20c_test.iloc[1], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 20'
                                                  )

In [ ]:
chart

## Route 99

In [ ]:
rt99_test = sanj_long_shapes.query('route_id == "99"')

In [ ]:
rt99_test[['name', 'route_short_name', 'route_long_name', 'shape_id']]

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt99_test.iloc[0], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 99')

In [ ]:
chart

In [ ]:
chart = chart_utils.flow_chart_from_shape_trip_row(rt99_test.iloc[1], stop_times=bus_st, ridership=source_ridership,
                              ridership_data_route='Rt 99')

In [ ]:
chart